In [1]:
import numpy as np
import pandas as pd 
import sklearn as sk
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
#pd.options.display.max_rows = None
#pd.options.display.max_columns = None

In [2]:
game = pd.read_csv('game_info.csv') 

In [3]:
display(game.head(1))
print (game.shape)

,id,slug,name,metacritic,released,tba,updated,website,rating,rating_top,...,developers,genres,publishers,esrb_rating,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing
0,1,dgeneration-hd,D/Generation HD,NaN,2015-10-23,False,2019-09-17T11:58:57,http://dgeneration.net,0.0,0,...,West Coast Software||West Coast Software Limited,Adventure||Puzzle,West Coast Software||West Coast Software Limited,Everyone 10+,3,70,0,2,0,0


(345667, 27)


In [4]:
game.isnull().sum()

id                           0
slug                         2
name                         2
metacritic              342428
released                 27201
tba                          0
updated                      0
website                 284812
rating                       0
rating_top                   0
playtime                     0
achievements_count           0
ratings_count                0
suggestions_count            0
game_series_count            0
reviews_count                0
platforms                 3391
developers                6236
genres                   67190
publishers              214606
esrb_rating             290551
added_status_yet             0
added_status_owned           0
added_status_beaten          0
added_status_toplay          0
added_status_dropped         0
added_status_playing         0
dtype: int64

# EDA

In [5]:
# Fill missing value 
#for i in range (0,game.shape[0]):
#    if pd.isnull(game['developers'][i]):
#        game['developers'][i] = game['publishers'][i]

In [6]:
# Dropping missing datas more than 80%
game.drop(['metacritic','esrb_rating','developers','publishers','website'],axis = 1,inplace = True)
# Dropping not meaningful data 
game.drop(['id','slug'],axis = 1,inplace = True)
# Dropping the game without name
game = game[~game['name'].isnull()].reset_index(drop=True)

In [7]:
# Replace the missing value with unknown instead of Null value 
game['genres'] = game['genres'].fillna('unknown')
game['released'] = game['released'].fillna('unknown')
#game['developers'] = game['developers'].fillna('unknown')
game['platforms'] = game['platforms'].fillna('unknown')

In [8]:
for i in range (0,len(game.isnull().sum())):
    if game.isnull().sum()[i] == 0:
        continue
    else:
        print('unclean')
print('clean')

clean


In [9]:
game['updated'] = pd.to_datetime(game['updated'])

In [10]:
game.head(10)

,name,released,tba,updated,rating,rating_top,playtime,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,genres,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing
0,D/Generation HD,2015-10-23,False,2019-09-17 11:58:57,0.0,0,1,80,0,283,0,0,PC||macOS||Xbox One||PlayStation 4||Nintendo S...,Adventure||Puzzle,3,70,0,2,0,0
1,G Prime Into The Rain,2016-01-06,False,2019-11-06 23:04:19,0.0,0,0,26,0,375,0,0,macOS||PC||Xbox One,Simulation||Indie,0,39,0,1,0,0
2,Land Sliders,2015-09-24,False,2019-10-22 13:56:16,0.0,0,0,0,1,71,0,1,iOS,Adventure||Arcade,0,0,1,1,1,0
3,Pixel Gear,2016-10-20,False,2019-08-28 22:16:02,0.0,0,0,0,0,443,0,0,PC||PlayStation 4,Action||Indie,0,0,0,2,0,0
4,Gods and Idols,2016-12-12,False,2019-09-17 13:37:13,0.0,0,1,0,4,268,0,4,PC,RPG||Strategy||Massively Multiplayer,1,57,0,1,0,0
5,Plague venue,2017-12-02,False,2019-01-09 12:41:06,0.0,0,0,0,0,45,0,0,Web,unknown,0,0,0,0,0,0
6,The Moon Sliver (itch),2014-05-03,False,2019-01-09 12:41:06,0.0,0,0,0,0,121,0,0,PC||macOS,Adventure,0,0,0,0,0,0
7,Red Entity,2014-08-26,False,2019-01-09 12:41:06,0.0,0,0,0,0,33,0,0,PC||macOS||Linux,Action||Shooter,0,0,0,0,0,0
8,Rollman (itch),2015-03-22,False,2019-01-09 12:41:06,0.0,0,0,0,0,131,0,0,PC||Linux,Adventure,0,0,0,0,0,0
9,HippiesVsCops,2016-04-18,False,2019-08-28 23:25:11,0.0,0,0,0,0,32,0,0,PC,Strategy,0,0,0,0,0,0


In [11]:
genres = game['genres'].str.get_dummies('|')

In [12]:
platforms = game['platforms'].str.get_dummies('|')

In [13]:
game['tba'] = game['tba'].map({False: '0',True:'1'})

In [14]:
game

,name,released,tba,updated,rating,rating_top,playtime,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,genres,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing
0,D/Generation HD,2015-10-23,0,2019-09-17 11:58:57,0.0,0,1,80,0,283,0,0,PC||macOS||Xbox One||PlayStation 4||Nintendo S...,Adventure||Puzzle,3,70,0,2,0,0
1,G Prime Into The Rain,2016-01-06,0,2019-11-06 23:04:19,0.0,0,0,26,0,375,0,0,macOS||PC||Xbox One,Simulation||Indie,0,39,0,1,0,0
2,Land Sliders,2015-09-24,0,2019-10-22 13:56:16,0.0,0,0,0,1,71,0,1,iOS,Adventure||Arcade,0,0,1,1,1,0
3,Pixel Gear,2016-10-20,0,2019-08-28 22:16:02,0.0,0,0,0,0,443,0,0,PC||PlayStation 4,Action||Indie,0,0,0,2,0,0
4,Gods and Idols,2016-12-12,0,2019-09-17 13:37:13,0.0,0,1,0,4,268,0,4,PC,RPG||Strategy||Massively Multiplayer,1,57,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345660,Holy or Dead,2017-05-17,0,2019-01-09 12:41:06,0.0,0,0,0,0,40,0,0,PC,unknown,0,0,0,0,0,0
345661,Airstrike HD Demo,2016-03-04,0,2019-01-09 12:41:06,0.0,0,0,0,0,188,0,0,PC,Action,0,0,0,0,0,0
345662,Urania's Mirror,2016-04-25,0,2019-01-09 12:41:06,0.0,0,0,0,0,60,0,0,PC,Adventure,0,0,0,0,0,0
345663,Simucities,2017-05-26,0,2019-01-09 12:41:06,0.0,0,0,0,0,73,0,0,PC,unknown,0,0,0,0,0,0
